In [73]:

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time

In [74]:
vid_1 = 'slow_traffic_small.mp4'
vid_2 = 'vtest.avi'

capture = cv.VideoCapture(vid_2)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + vid_2)
    exit(0)

Método Naive

In [75]:
naive_total_elapsed = []

def calculate_background(frames): 
    start = time.time()
   
    new_mask = np.median(frames, axis=0)

    end = time.time()
    elapsed = end - start
    naive_total_elapsed.append(elapsed)
    return new_mask.astype(np.uint8)
                


Método KNN

In [76]:
open_cv_times = []    

metodo = 'KNN'
if metodo == 'KNN':
    backSub = cv.createBackgroundSubtractorKNN()



In [77]:
def substract_backgorund(frames_count, capture, frame_spaces=35):
    ret, frame = capture.read()
    i = 0
    FRAMES = frames_count
    frames_to_add = FRAMES - 1
    frames = np.zeros((FRAMES, frame.shape[0], frame.shape[1], frame.shape[2]))
    should_add_frames = True
    add_frames_step = True
    frames_added = 0
    background = frames[0,:,:,:]
    first_frames = True
    while True:
        # Leemos frames
        ret, frame = capture.read()
        if frame is None:
            break
        
        if first_frames:
            if frames_added < FRAMES:
                frames[frames_added] = frame
                frames_added = frames_added + 1
                background = calculate_background(frames)
            else:
                first_frames = False
                frames_added = 0

        if i % frame_spaces == 0 and not first_frames:
            if frames_added < frames_to_add:
                frames[frames_added] = frame
                frames_added = frames_added + 1
                background = calculate_background(frames)
            else:
                frames[frames_added] = background
                frames_added = 0
            
        i = i + 1
  
        new_frame = cv.subtract(background, frame)
        new_frame[new_frame >30] = 255

        # Aplico método de opencv y mido el tiempo
        opencv_start = time.time()
        fgMask = backSub.apply(frame)
        opencv_end = time.time()
        opencv_elapsed = opencv_end - opencv_start
        open_cv_times.append(opencv_elapsed)

        cv.imshow('Frames', frame)
        cv.imshow('Método Naive', new_frame)
        cv.imshow('Fondo', background)
        cv.imshow('FG Mask', fgMask)

        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break
        
    cv.destroyAllWindows()
    capture.release()

# Implementación

In [78]:
substract_backgorund(5, capture, 35)

# Conclusiones

Tiempo promedio algoritmo OpenCV vs Naive

In [79]:
avg_naive = np.average(naive_total_elapsed)*1000 # seconds to ms
print("Método Naive: {:.6f}".format(avg_naive))
avg_opencv = np.average(open_cv_times)*1000 # seconds to ms
print("Método OpenCV:  {:.6f}".format(avg_opencv))

Método Naive: 143.699967
Método OpenCV:  7.446103


Como se puede observar, el algoritmo de Naive es mucho más lento q el de OpenCV.

Ahora vemos la cantidad de calculos de fondos de ambos algoritmos.

In [80]:
print("Método Naive:  " + str(len(naive_total_elapsed)))
print("Método OpenCV:  " + str(len(open_cv_times)))

Método Naive:  23
Método OpenCV:  794


El método de OpenCV se ejecuta muchas más veces que el Naive.

Por último vemos la suma de los tiempos en los qe se ejecutaron ambos algoritmos.

In [81]:
sum_naive = np.sum(naive_total_elapsed)*1000 # seconds to ms
sum_opencv = np.sum(open_cv_times)*1000 # seconds to ms

print("Suma de tiempo método Naive: {:.6f}".format(sum_naive))
print("Suma de tiempo método OpenCV:  {:.6f}".format(sum_opencv))

Suma de tiempo método Naive: 3305.099249
Suma de tiempo método OpenCV:  5912.206173
